# Introduction à Transformer-lens

## Configuration

Ce fragment sert à configurer Plotly pour que ses graphiques interactifs puissent s’afficher correctement dans l’environnement d’exécution actuel (VSCode, Jupyter Notebook, etc.).

* import plotly.io as pio : on importe le module qui gère les paramètres d’entrée/sortie de Plotly.
* pio.renderers.default = "notebook_connected" : Plotly peut utiliser plusieurs « moteurs de rendu » (par exemple browser, colab, notebook_connected, etc.) selon l’endroit où tourne le code. Ici on force l’utilisation de notebook_connected, qui est adapté aux notebooks locaux (Jupyter/VSCode) et permet d’afficher les graphiques directement dans la cellule.
* print(...) : on affiche quel renderer est activé, histoire de vérifier que la configuration est bien appliquée.

En résumé, ce code évite les problèmes fréquents d’affichage (par exemple un graphique qui ne s’ouvre pas, ou qui tente de s’ouvrir dans un navigateur externe) en fixant explicitement le moteur de rendu approprié à l’environnement.

In [1]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

Using renderer: notebook_connected


Ce bout de code utilise circuitsvis, une petite librairie de visualisation interactive souvent employée avec TransformerLens.

* import circuitsvis as cv : on importe la librairie.
* cv.examples.hello("Nils") : appelle une fonction d’exemple intégrée. C’est juste une demo de test qui affiche une petite visualisation interactive (sous forme d’HTML/Javascript), par exemple une boîte avec “Hello Nils 👋”.

En pratique :

* Dans un notebook Jupyter/VSCode, la sortie apparaît directement sous la cellule.
* Dans Colab, ça peut nécessiter d’installer circuitsvis[colab] et d’activer le mode Colab renderer (cv.notebook.init()).

C’est donc simplement un “hello world” graphique fourni par circuitsvis pour vérifier que l’intégration HTML fonctionne.

In [3]:
import circuitsvis as cv
cv.examples.hello("Artificialis")

Ce bloc est un préambule classique : on rassemble toutes les librairies dont on aura besoin pour manipuler des tenseurs, définir des réseaux, visualiser et écrire du code plus lisible.

* PyTorch : sert à créer et entraîner des modèles de deep learning.
* torch.nn contient les briques de réseaux de neurones (layers, fonctions d’activation, etc.).
* einops simplifie les réarrangements de tenseurs avec une syntaxe lisible (rearrange, reduce, etc.).
* fancy_einsum est une version plus expressive de torch.einsum, avec des noms de dimensions explicites → facilite la lecture et évite les erreurs d’indices.
* tqdm affiche des barres de progression (l’alias auto choisit le meilleur backend selon l’environnement : notebook, terminal, etc.).
* plotly.express : création rapide de graphiques interactifs (scatter, heatmap, etc.).
* jaxtyping : permet d’annoter les types de tenseurs (Float[Tensor, "batch d_model"], par ex.) pour mieux documenter et vérifier le code.
* partial : utilitaire Python qui fixe certains arguments d’une fonction pour créer une variante simplifiée.

In [4]:
# Import stuff
import torch
import torch.nn as nn
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import plotly.express as px

from jaxtyping import Float, Int
from functools import partial

from typing import Optional, Any

import numpy as np

In [5]:
# import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, FactoredMatrix

In [6]:
torch.set_grad_enabled(False)

torch.autograd.grad_mode.set_grad_enabled(mode=False)

In [7]:
def imshow(
        tensor: Float[np.ndarray, "height width"],  # Matrice 2D (ex: heatmap)
        renderer: Optional[str] = None,             # Nom du renderer Plotly (ex: "notebook_connected")
        xaxis: str = "",                            # Label axe X
        yaxis: str = "",                            # Label axe Y
        **kwargs: Any                               # Options supplémentaires passées à px.imshow
) -> None:
    """
    Affiche un tenseur 2D sous forme d'image (heatmap) avec Plotly.

    Args:
        tensor: tableau 2D (PyTorch/JAX/NumPy) représentant des valeurs à colorer.
        renderer: moteur d'affichage Plotly (None = valeur par défaut).
        xaxis: étiquette de l'axe X.
        yaxis: étiquette de l'axe Y.
        **kwargs: options supplémentaires (ex: title="Mon Graphique").
    """
    px.imshow(
        utils.to_numpy(tensor),
        color_continuous_midpoint=0.0,
        color_continuous_scale="RdBu",
        labels={"x":xaxis, "y":yaxis},
        **kwargs
    ).show(renderer)
# end imshow

def line(
        tensor: Float[np.ndarray, "n"],              # Vecteur 1D (courbe)
        renderer: Optional[str] = None,
        xaxis: str = "",
        yaxis: str = "",
        **kwargs: Any
) -> None:
    """
    Affiche un tenseur 1D sous forme de courbe.

    Args:
        tensor: tableau 1D (ex: évolution d'une valeur au cours du temps).
        renderer: moteur d'affichage Plotly.
        xaxis: étiquette de l'axe X.
        yaxis: étiquette de l'axe Y.
        **kwargs: options supplémentaires pour px.line.
    """
    px.line(
        utils.to_numpy(tensor),
        labels={"x":xaxis, "y":yaxis},
        **kwargs
    ).show(renderer)
# end line

def scatter(
        x: Float[np.ndarray, "n"],                   # Coordonnées X (1D)
        y: Float[np.ndarray, "n"],                   # Coordonnées Y (1D)
        xaxis: str = "",
        yaxis: str = "",
        caxis: str = "",                             # Légende pour la couleur
        renderer: Optional[str] = None,
        **kwargs: Any
) -> None:
    """
    Affiche un nuage de points (scatter plot).

    Args:
        x: tableau 1D pour les abscisses.
        y: tableau 1D pour les ordonnées.
        xaxis: étiquette de l'axe X.
        yaxis: étiquette de l'axe Y.
        caxis: nom de la légende associée à la couleur.
        renderer: moteur d'affichage Plotly.
        **kwargs: options supplémentaires pour px.scatter (ex: color=...).
    """
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(
        y=y,
        x=x,
        labels={"x":xaxis, "y":yaxis, "color":caxis},
        **kwargs
    ).show(renderer)
# end scatter

## Introduction

Ceci est un notebook de démonstration pour [TransformerLens](https://github.com/TransformerLensOrg/TransformerLens), **une bibliothèque que j’ai ([Neel Nanda](https://neelnanda.io)) écrite pour faire de l’[interprétabilité mécanistique](https://distill.pub/2020/circuits/zoom-in/) de modèles de langage de type GPT-2.**

L’objectif de l’interprétabilité mécanistique est de prendre un modèle entraîné et de rétro-concevoir les algorithmes que le modèle a appris durant son entraînement à partir de ses poids. Le fait est qu’aujourd’hui, nous avons des programmes capables de parler anglais à un niveau humain (GPT-3, PaLM, etc.), mais nous n’avons aucune idée de leur fonctionnement ni de comment en écrire un nous-mêmes. Cela me choque profondément, et je veux résoudre ce problème ! L’interprétabilité mécanistique est un domaine encore très jeune et très réduit, avec *beaucoup* de problèmes ouverts — si vous voulez contribuer, essayez-en un ! **Si vous voulez vous former, consultez [mon guide pour débuter](https://neelnanda.io/getting-started), et si vous voulez plonger directement dans un problème ouvert, regardez ma série [200 problèmes concrets ouverts en interprétabilité mécanistique](https://neelnanda.io/concrete-open-problems).**

J’ai écrit cette bibliothèque parce qu’après avoir quitté l’équipe d’interprétabilité d’Anthropic et commencé à faire de la recherche indépendante, j’ai été extrêmement frustré par l’état des outils open source. Il existe beaucoup d’infrastructures excellentes comme HuggingFace et DeepSpeed pour *utiliser* ou *entraîner* des modèles, mais très peu pour explorer leurs entrailles et rétro-concevoir leur fonctionnement. **Cette bibliothèque cherche à combler ce manque**, et à rendre l’accès au domaine facile, même si l’on ne travaille pas dans une organisation industrielle dotée d’une vraie infrastructure ! Les fonctionnalités principales se sont beaucoup inspirées de l’[excellent outil Garcon d’Anthropic](https://transformer-circuits.pub/2021/garcon/index.html). Merci à Nelson Elhage et Chris Olah d’avoir construit Garcon et de m’avoir montré la valeur d’une bonne infrastructure pour accélérer la recherche exploratoire !

Le principe de conception central que j’ai suivi est de favoriser l’analyse exploratoire : l’un des aspects les plus amusants de l’interprétabilité mécanistique, comparée au machine learning classique, est la boucle de rétroaction extrêmement courte ! L’objectif de cette bibliothèque est de réduire au maximum l’écart entre avoir une idée d’expérience et en voir les résultats, afin que **la recherche ressemble à un jeu** et permette d’entrer dans un état de flux. Ce notebook montre comment la bibliothèque fonctionne et comment l’utiliser, mais si vous voulez voir à quel point elle est adaptée à la recherche exploratoire, regardez [mon notebook d’analyse sur l’Indirect Objection Identification](https://neelnanda.io/exploratory-analysis-demo) ou [mon enregistrement de moi-même en train de faire de la recherche](https://www.youtube.com/watch?v=yo4QvDn-vsU) !


## Chargement et exécution des modèles

TransformerLens est fourni avec plus de 40 modèles open source de type GPT. Vous pouvez en charger n’importe lequel avec la commande `HookedTransformer.from_pretrained(MODEL_NAME)`.

Dans ce notebook de démonstration, nous allons utiliser **GPT-2 Small**, un modèle de 80 millions de paramètres. Pour les autres modèles disponibles, référez-vous à la section *Available Models*.

On commence par récupérer le device utilisé pour le calcul.

In [8]:
device = utils.get_device()

On charge ensuite un modèle pré-entraîné depuis HuggingFace.

In [9]:
# NBVAL_IGNORE_OUTPUT
model = HookedTransformer.from_pretrained("gpt2-small", device=device)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-small into HookedTransformer


Pour tester le modèle, calculons la *loss* sur ce texte !
Les modèles peuvent être exécutés soit sur une **chaîne de caractères**, soit sur un **tenseur de tokens** (forme : `[batch, position]`, uniquement des entiers).

Les types de sortie possibles sont :

* `"logits"` — tenseur de forme `[batch, position, d_vocab]` (valeurs flottantes) ;
* `"loss"` — la *cross-entropy loss* lors de la prédiction du prochain token ;
* `"both"` — un tuple `(logits, loss)` ;
* `None` — exécuter le modèle sans calculer les logits (plus rapide si l’on ne veut que les activations intermédiaires).


In [13]:
model_description_text = """## Loading Models

HookedTransformer comes loaded with >40 open source GPT-style models. You can load any of them in with `HookedTransformer.from_pretrained(MODEL_NAME)`. See my explainer for documentation of all supported models, and this table for hyper-parameters and the name used to load them. Each model is loaded into the consistent HookedTransformer architecture, designed to be clean, consistent and interpretability-friendly.

For this demo notebook we'll look at GPT-2 Small, an 80M parameter model. To try the model the model out, let's find the loss on this paragraph!

For this demo notebook we'll look at GPT-2 Small, an 80M parameter model. To try the model the model out, let's find the loss on this paragraph!"""
logits, loss = model(model_description_text, return_type="both")
print("Model loss:", loss)
print("Model logits:", logits.size())

Model loss: tensor(3.3063, device='cuda:0')
Model logits: torch.Size([1, 178, 50257])


On transforme les logits en probabilités (probabilités d'être le prochain token pour chaque entrée du vocabulaire). Puis on détermine quel est le plus probable grâce à argmax.

In [22]:
import torch.nn.functional as F
probs = F.softmax(logits, dim=-1)
torch.argmax(probs[0, -1])

tensor(1114, device='cuda:0')

## Mettre en cache toutes les activations

La première opération de base en interprétabilité mécanistique consiste à **ouvrir la boîte noire du modèle** et à examiner toutes ses activations internes. On peut le faire avec `logits, cache = model.run_with_cache(tokens)`. Testons cela sur la première ligne du résumé de l’article GPT-2.

#### À propos de `remove_batch_dim`

Toutes les activations à l’intérieur du modèle commencent par une dimension de batch. Ici, comme on ne passe qu’un seul batch, cette dimension vaut toujours 1 et peut être pénible à manipuler ; en passant l’argument `remove_batch_dim=True`, on la supprime. L’instruction `gpt2_cache_no_batch_dim = gpt2_cache.remove_batch_dim()` aurait eu le même effet.


On commence par définir le texte qu'on va donner à GPT-2.

In [23]:
gpt2_text = "Natural language processing tasks, such as question answering, machine translation, reading comprehension, and summarization, are typically approached with supervised learning on task specific datasets."

On transforme ce texte en token, c'est à dire en ID (int) qui indique la place du token dans le vocabulaire.
Cette liste d'ID a une taille de (1, 32). Pour 1 batch, et 32 tokens.

In [27]:
gpt2_tokens = model.to_tokens(gpt2_text)

torch.Size([1, 32])

On donne les tokens à GPT-2 mais on garde en cache les activations. On récupère les prédictions (gpt2_logits) et le cache (gpt2_cache).

In [ ]:
gpt2_logits, gpt2_cache = model.run_with_cache(gpt2_tokens, remove_batch_dim=True)

gpt2_logits a une taille de (1, 32, 50257). Un batch, 32 tokens, et 50257 la taille du vocabulaire. gpt2_cache est lui un objet [ActivationCache](https://transformerlensorg.github.io/TransformerLens/generated/code/transformer_lens.ActivationCache.html).

## Visualiser les pattern d'attention

Visualisons le **pattern d’attention** de toutes les têtes de la couche 0, en utilisant la bibliothèque [CircuitsVis d’Alan Cooney](https://github.com/alan-cooney/CircuitsVis) (basée sur la [bibliothèque PySvelte d’Anthropic](https://github.com/anthropics/PySvelte)).

Nous examinons le pattern d’attention dans `gpt2_cache`, un objet `ActivationCache`, en donnant le nom de l’activation suivi de l’indice de couche (ici, l’activation s’appelle `"attn"` et l’indice de couche est `0`). Sa forme est `[head_index, destination_position, source_position]`. Nous utilisons la méthode `model.to_str_tokens` pour convertir le texte en une liste de tokens (chaînes de caractères), puisqu’il existe un poids d’attention entre chaque paire de tokens.

Cette visualisation est **interactive** ! Passez la souris sur un token ou une tête, et cliquez pour verrouiller. La grille en haut à gauche et, pour chaque tête, représente le pattern d’attention sous forme de matrice « position de destination × position source ». Elle est **triangulaire inférieure** parce que GPT-2 utilise une **attention causale** : l’attention ne peut regarder que vers le passé, donc l’information ne peut circuler que vers l’avant dans le réseau.

Consultez la section *ActivationCache* pour en savoir plus sur ce que `gpt2_cache` permet de faire.

In [35]:
# Couche 0 de GPT-2
attention_pattern = gpt2_cache["pattern", 0, "attn"]
gpt2_str_tokens = model.to_str_tokens(gpt2_text)

`attention_pattern` fait (12, 32, 32), pour 12 tête d'attention, 32 tokens d'origine, 32 tokens de destination. `gpt2_str_tokens` est une liste de string qui sont les tokens de la chaîne `gpt2_text`.

In [36]:
print("Layer 0 Head Attention Patterns:")
cv.attention.attention_patterns(tokens=gpt2_str_tokens, attention=attention_pattern)

Layer 0 Head Attention Patterns:


Dans ce cas, nous ne voulions que les patterns d’attention de la couche 0, mais nous stockons les activations internes provenant de tous les endroits du modèle. C’est pratique d’avoir accès à toutes les activations, mais cela peut devenir très coûteux en mémoire pour des modèles plus grands, des batchs plus volumineux ou des séquences plus longues.

De plus, nous n’avons pas besoin d’exécuter l’entier du passage avant (full forward pass) dans le modèle pour récupérer les patterns d’attention de la couche 0. La cellule suivante ne collectera que les patterns d’attention de la couche 0 et arrêtera la passe avant à la couche 1, ce qui réduit fortement l’empreinte mémoire et le calcul nécessaires.

In [38]:
# Nom du hook
attn_hook_name = "blocks.0.attn.hook_pattern"

# Layer correspondant
attn_layer = 0

# On garde en cache jusqu'à la couche < 1, et on garde seulement le hook.
_, gpt2_attn_cache = model.run_with_cache(gpt2_tokens, remove_batch_dim=True, stop_at_layer=attn_layer + 1, names_filter=[attn_hook_name])

# Récupère les cartes d'attention
gpt2_attn = gpt2_attn_cache[attn_hook_name]
assert torch.equal(gpt2_attn, attention_pattern)

## Hooks: Intervening on Activations

L’un des grands avantages de l’interprétation des réseaux de neurones est que nous avons un **contrôle total** sur notre système. D’un point de vue computationnel, nous savons exactement quelles opérations ont lieu à l’intérieur (même si nous ne savons pas toujours ce qu’elles signifient !). Et nous pouvons effectuer des modifications précises et ciblées, puis observer comment le comportement du modèle et ses activations internes changent. C’est un outil extrêmement puissant, car il permet par exemple de mettre en place des contre-factuels rigoureux et des interventions causales pour comprendre facilement le comportement du modèle.

En conséquence, la possibilité de faire cela constitue une opération **centrale**, et c’est l’une des principales fonctionnalités offertes par **TransformerLens** ! La clé ici est le concept de **hook points**. Chaque activation à l’intérieur du transformeur est entourée d’un hook point, qui permet de la modifier ou d’intervenir dessus.

Nous procédons en ajoutant une **fonction hook** à cette activation. La fonction hook prend en entrée `current_activation_value, hook_point` et retourne `new_activation_value`. Lors de l’exécution du modèle, l’activation est calculée normalement, puis la fonction hook est appliquée pour produire une valeur de remplacement, qui est insérée à la place de l’activation. La fonction hook peut être n’importe quelle fonction Python, tant qu’elle retourne un tenseur de la bonne forme.

#### Lien avec les hooks PyTorch

Les [hooks PyTorch](https://blog.paperspace.com/pytorch-hooks-gradient-clipping-debugging/) sont une fonctionnalité très utile mais souvent méconnue… et parfois assez bancale. Ils permettent d’agir sur une couche et de modifier son entrée ou sa sortie, ou encore le gradient lors de l’autodifférentiation. La différence essentielle est que les **hook points** agissent sur les *activations* et non sur les couches. Cela signifie que l’on peut intervenir à l’intérieur d’une couche, directement sur chaque activation, sans se soucier de la structure précise des couches du transformeur. Et il est immédiatement clair de quelle manière l’effet du hook s’applique. Cette approche a été directement inspirée par l’usage des *ProbePoints* dans [Garcon](https://transformer-circuits.pub/2021/garcon/index.html).

Ils s’accompagnent aussi de plusieurs améliorations pratiques, comme la méthode `model.reset_hooks()` qui permet de supprimer tous les hooks, ou encore des méthodes utilitaires pour ajouter temporairement des hooks le temps d’un seul passage avant (*forward pass*). Avec les hooks PyTorch standards, il est *extrêmement* facile de se tirer une balle dans le pied !


Comme exemple de base, **ablons** (mettons à zéro) la tête 7 de la couche 0 sur le texte ci-dessus.

On définit une fonction `head_ablation_hook`. Elle prend le tenseur de valeurs (value) pour la couche d’attention 0, met à zéro la composante où `head_index == 7`, puis la renvoie (Note : on renvoie par convention, mais comme on modifie l’activation *in place*, ce n’est pas strictement *nécessaire*).

On utilise ensuite le helper `run_with_hooks` pour exécuter le modèle et ajouter *temporairement* ce hook uniquement pour ce passage. On fournit le hook sous forme de tuple : le nom de l’activation (qui est aussi le nom du hook point — récupérable avec `utils.get_act_name`) et la fonction hook.


In [39]:
layer_to_ablate = 0
head_index_to_ablate = 8

# We define a head ablation hook
# The type annotations are NOT necessary, they're just a useful guide to the reader
def head_ablation_hook(
    value: Float[torch.Tensor, "batch pos head_index d_head"],
    hook: HookPoint
) -> Float[torch.Tensor, "batch pos head_index d_head"]:
    """
    Define an ablation hook.
    """
    # Value est de taille (1 batch, 33 tokens ?, 12 têtes, 64 ?)
    value[:, :, head_index_to_ablate, :] = 0.
    return value
# end head_ablation_hook

# Tourne le modèle sans hook
original_loss = model(gpt2_tokens, return_type="loss")

# Tourne le modèle avec le hook
ablated_loss = model.run_with_hooks(
    gpt2_tokens,
    return_type="loss",
    fwd_hooks=[
        (
            utils.get_act_name("v", layer_to_ablate),
            head_ablation_hook
        )
    ]
)

# Affiche
print(f"Original Loss: {original_loss.item():.3f}")
print(f"Ablated Loss: {ablated_loss.item():.3f}")

Original Loss: 3.624
Ablated Loss: 4.983


**Attention :** Les hooks sont un **état global** — ils sont ajoutés au modèle et y restent tant qu’on ne les a pas retirés. La fonction `run_with_hooks` essaie d’abstraire cela en les traitant comme un état local, en supprimant tous les hooks à la fin de l’exécution. Mais on peut facilement se tirer une balle dans le pied si, par exemple, une erreur survient dans l’un des hooks et que la fonction ne termine jamais.

Si vous commencez à rencontrer des bugs, essayez `model.reset_hooks()` pour tout nettoyer. Par ailleurs, si vous **ajoutez volontairement vos propres hooks** que vous souhaitez conserver, vous pouvez le faire avec `add_perma_hook` sur le *HookPoint* concerné.

### *Activation Patching* sur la tâche d’Identification de l’Objet Indirect

Pour un exemple un peu plus élaboré, utilisons des hooks pour appliquer le **[activation patching](https://dynalist.io/d/n2ZWtnoYHrU1s4vnFSAQ519J#z=qeWBvs-R-taFfcCq-S_hgMqx)** sur la tâche **[Indirect Object Identification](https://dynalist.io/d/n2ZWtnoYHrU1s4vnFSAQ519J#z=iWsV3s5Kdd2ca3zNgXr5UPHa)** (IOI).

La tâche IOI consiste à reconnaître que, dans une phrase comme « After John and Mary went to the store, Mary gave a bottle of milk to », la suite correcte est « John » plutôt que « Mary » (c’est-à-dire identifier l’objet indirect). Redwood Research a publié [un excellent article étudiant le circuit sous-jacent dans GPT-2 Small](https://arxiv.org/abs/2211.00593).

L’**activation patching** est une technique issue de l’[excellent papier ROME de Kevin Meng et David Bau](https://rome.baulab.info/). L’objectif est d’identifier quelles activations du modèle sont importantes pour accomplir une tâche. Pour cela, on prépare un **prompt propre (clean)** et un **prompt corrompu (corrupted)** ainsi qu’une **métrique** de performance. On choisit ensuite une activation précise du modèle, on exécute le modèle sur le prompt corrompu, puis on *intervient* sur cette activation en la **remplaçant** par sa valeur obtenue lorsque le modèle est exécuté sur le prompt propre. On applique alors la métrique et on mesure dans quelle mesure ce « patch » restaure la performance du prompt propre.
(Voir [une démonstration plus détaillée de l’activation patching ici](https://colab.research.google.com/github/TransformerLensOrg/TransformerLens/blob/main/demos/Exploratory_Analysis_Demo.ipynb).)


Ici, notre **prompt propre** est :
« After John and Mary went to the store, **Mary** gave a bottle of milk to » et notre **prompt corrompu** est :
« After John and Mary went to the store, **John** gave a bottle of milk to »

Notre métrique est la différence entre le logit correct (*John*) et le logit incorrect (*Mary*) sur le dernier token.

On observe que la différence de logit est nettement positive pour le prompt propre, et nettement négative pour le prompt corrompu, ce qui montre que le modèle est bel et bien capable de réaliser la tâche !

In [40]:
# Prompts clean et corrompus
clean_prompt = "After John and Mary went to the store, Mary gave a bottle of milk to"
corrupted_prompt = "After John and Mary went to the store, John gave a bottle of milk to"

# Transforme les prompts en token
clean_tokens = model.to_tokens(clean_prompt)
corrupted_tokens = model.to_tokens(corrupted_prompt)

# La métrique d'erreur
def logits_to_logit_diff(
        logits,
        correct_answer=" John",
        incorrect_answer=" Mary"
):
    # model.to_single_token maps a string value of a single token to the token index for that token
    # If the string is not a single token, it raises an error.
    correct_index = model.to_single_token(correct_answer)
    incorrect_index = model.to_single_token(incorrect_answer)
    return logits[0, -1, correct_index] - logits[0, -1, incorrect_index]
# end logits_to_logit_diff

# On fait tourner le modèle avec le prompt clean pour garder les activations.
clean_logits, clean_cache = model.run_with_cache(clean_tokens)
clean_logit_diff = logits_to_logit_diff(clean_logits)
print(f"Clean logit difference: {clean_logit_diff.item():.3f}")

# On a pas besoin de garder les activations sur ce coup.
corrupted_logits = model(corrupted_tokens)
corrupted_logit_diff = logits_to_logit_diff(corrupted_logits)
print(f"Corrupted logit difference: {corrupted_logit_diff.item():.3f}")

Clean logit difference: 4.276
Corrupted logit difference: -2.738


Nous allons maintenant définir la fonction de *hook* pour faire de l’**activation patching**. Ici, nous allons **patcher le flux résiduel** (*residual stream*) au **début d’une couche donnée** et à **une position donnée**. Cela nous permettra de mesurer dans quelle mesure le modèle utilise le flux résiduel, à cette couche et à cette position, pour représenter l’information clé de la tâche.

Comme nous voulons itérer sur **toutes les couches** et **toutes les positions**, nous écrivons le hook de façon à accepter un paramètre `position`. Les fonctions de hook doivent avoir la signature d’entrée `(activation, hook)`, mais on peut utiliser `functools.partial` pour fixer à l’avance le paramètre `position` avant de passer la fonction à `run_with_hooks`.


In [41]:
# On définit un hook pour patcher le residual.
# On choisit d'agir sur le résidual au début de la couche, donc resid_pre.
def residual_stream_patching_hook(
    resid_pre: Float[torch.Tensor, "batch pos d_model"],
    hook: HookPoint,
    position: int
) -> Float[torch.Tensor, "batch pos d_model"]:
    """
    To document.
    """
    # Each HookPoint has a name attribute giving the name of the hook.
    clean_resid_pre = clean_cache[hook.name]
    resid_pre[:, position, :] = clean_resid_pre[:, position, :]
    return resid_pre
# end residual_stream_patching_hook

# On crée un tensor pour garder les résultats de chaque patch.
# On le met sur le device pour éviter des problèmes GPU/CPU.
num_positions = len(clean_tokens[0])
ioi_patching_result = torch.zeros((model.cfg.n_layers, num_positions), device=model.cfg.device)

# Pour chaque couche
for layer in tqdm.tqdm(range(model.cfg.n_layers)):
    # Pour chaque position
    for position in range(num_positions):
        # Use functools.partial to create a temporary hook function with the position fixed
        temp_hook_fn = partial(residual_stream_patching_hook, position=position)
        # Run the model with the patching hook
        patched_logits = model.run_with_hooks(corrupted_tokens, fwd_hooks=[
            (utils.get_act_name("resid_pre", layer), temp_hook_fn)
        ])
        # Calculate the logit difference
        patched_logit_diff = logits_to_logit_diff(patched_logits).detach()
        # Store the result, normalizing by the clean and corrupted logit difference so it's between 0 and 1 (ish)
        ioi_patching_result[layer, position] = (patched_logit_diff - corrupted_logit_diff)/(clean_logit_diff - corrupted_logit_diff)
    # end for position
# end for layer

  0%|          | 0/12 [00:00<?, ?it/s]

Nous pouvons maintenant visualiser les résultats et constater que ce calcul est extrêmement localisé dans le modèle. Au départ, c’est le token du deuxième sujet (Mary) qui importe (logiquement, puisque c’est le seul token qui diffère). Toute l’information pertinente reste concentrée à cet endroit, jusqu’à ce que des têtes dans les couches 7 et 8 transfèrent cette information vers le dernier token, où elle est utilisée pour prédire l’objet indirect.

(Remarque – les têtes sont bien en couches 7 et 8, et non en 8 et 9, car nous avons patché le flux résiduel au début de chaque couche.)

In [42]:
# Add the index to the end of the label, because plotly doesn't like duplicate labels
token_labels = [f"{token}_{index}" for index, token in enumerate(model.to_str_tokens(clean_tokens))]
imshow(
    ioi_patching_result,
    x=token_labels,
    xaxis="Position",
    yaxis="Layer",
    title="Normalized Logit Difference After Patching Residual Stream on the IOI Task"
)

## Hooks : accéder aux activations

Les hooks peuvent aussi servir simplement à **accéder** à une activation — c’est-à-dire exécuter une fonction utilisant la valeur de cette activation, *sans* la modifier. Pour cela, il suffit que le hook **ne retourne rien** et qu’il n’édite pas l’activation en place.

C’est utile par exemple pour extraire des activations dans le cadre d’une tâche donnée, ou pour effectuer un calcul long sur de nombreux inputs, comme chercher le texte qui active le plus un neurone spécifique.
*(Remarque – tout ce que l’on peut faire ainsi *pourrait* aussi être fait avec `run_with_cache` suivi d’un post-traitement, mais ce flux de travail peut être plus intuitif et plus économe en mémoire.)*

Pour illustrer cela, examinons les **[induction heads](https://transformer-circuits.pub/2022/in-context-learning-and-induction-heads/index.html)** dans GPT-2 Small.

Les circuits d’induction sont des circuits très importants dans les modèles de langage génératifs : ils servent à détecter et à prolonger des sous-séquences répétées. Ils sont constitués de deux têtes situées dans des couches distinctes qui se combinent :

* une **tête “previous token”** qui regarde toujours le token précédent,
* une **tête d’induction** qui regarde le token *suivant* une occurrence antérieure du token courant.

Pourquoi est-ce important ? Prenons l’exemple d’un modèle qui doit prédire le token suivant dans un article de presse parlant de Michael Jordan. Le token `" Michael"`, en général, peut être suivi de nombreux noms de famille. Mais une tête d’induction va chercher, à partir de cette occurrence de `" Michael"`, le token situé juste après les occurrences précédentes de `" Michael"`, c’est-à-dire `" Jordan"`, et peut prédire avec assurance que c’est ce qui suivra.

Un fait intéressant concernant les têtes d’induction est qu’elles se **généralisent** à n’importe quelle séquence de tokens répétés. On peut le voir en générant des séquences de 50 tokens aléatoires, répétées deux fois, puis en traçant la perte moyenne de prédiction du token suivant en fonction de la position. On constate que le modèle passe de très mauvais à très performant **au point médian de la séquence**.


In [43]:
batch_size = 10
seq_len = 50
size = (batch_size, seq_len)

# Entrée aléatoire
input_tensor = torch.randint(1000, 10000, size)

# Sur le device
random_tokens = input_tensor.to(model.cfg.device)

# ???
repeated_tokens = einops.repeat(random_tokens, "batch seq_len -> batch (2 seq_len)")

# Donne ça au modèle
repeated_logits = model(repeated_tokens)

# Calcul le loss
correct_log_probs = model.loss_fn(repeated_logits, repeated_tokens, per_token=True)

# ???
loss_by_position = einops.reduce(correct_log_probs, "batch position -> position", "mean")

# ???
line(
    loss_by_position,
    xaxis="Position",
    yaxis="Loss",
    title="Loss by position on random repeated tokens"
)

Les têtes d’induction vont prêter attention depuis la **deuxième occurrence** de chaque token vers le token qui suit sa **première occurrence**, c’est-à-dire le token situé `50 - 1 == 49` positions en arrière. Ainsi, en observant l’**attention moyenne portée 49 tokens en arrière**, nous pouvons identifier les têtes d’induction ! Définissons un hook pour faire cela.

* On attache le hook à l’activation du **pattern d’attention**. Il y a un grand tenseur de pattern par couche, empilant toutes les têtes, donc on doit manipuler le tenseur pour obtenir un score par tête.
* Les fonctions de hook peuvent accéder à l’état global, donc on crée un grand tenseur pour stocker le score d’induction de chaque tête, et on ajoute simplement le score de chaque tête à la bonne position dans ce tenseur.
* Pour obtenir une fonction de hook unique qui fonctionne pour chaque couche, on utilise la méthode `hook.layer()` afin de récupérer l’indice de la couche (en interne, cela est inféré à partir des noms des hook points).
* Comme on veut attacher cela à **tous** les hook points de pattern d’attention, plutôt que de donner une chaîne de nom d’activation, on fournit cette fois un **name filter**. C’est une fonction booléenne appliquée aux noms des hook points, et le hook sera ajouté à tous ceux pour lesquels la fonction renvoie vrai.
  * `run_with_hooks` permet d’entrer une liste de paires `(act_name, hook_function)` à ajouter d’un coup, donc on aurait aussi pu le faire en donnant une liste avec un hook spécifique pour chaque couche.


In [42]:
# We make a tensor to store the induction score for each head. We put it on the model's device to avoid needing to move things between the GPU and CPU, which can be slow.
induction_score_store = torch.zeros((model.cfg.n_layers, model.cfg.n_heads), device=model.cfg.device)
def induction_score_hook(
    pattern: Float[torch.Tensor, "batch head_index dest_pos source_pos"],
    hook: HookPoint,
):
    # We take the diagonal of attention paid from each destination position to source positions seq_len-1 tokens back
    # (This only has entries for tokens with index>=seq_len)
    induction_stripe = pattern.diagonal(dim1=-2, dim2=-1, offset=1-seq_len)
    # Get an average score per head
    induction_score = einops.reduce(induction_stripe, "batch head_index position -> head_index", "mean")
    # Store the result.
    induction_score_store[hook.layer(), :] = induction_score

# We make a boolean filter on activation names, that's true only on attention pattern names.
pattern_hook_names_filter = lambda name: name.endswith("pattern")

model.run_with_hooks(
    repeated_tokens,
    return_type=None, # For efficiency, we don't need to calculate the logits
    fwd_hooks=[(
        pattern_hook_names_filter,
        induction_score_hook
    )]
)

imshow(induction_score_store, xaxis="Head", yaxis="Layer", title="Induction Score by Head")

La tête 5 de la couche 5 obtient un score extrêmement élevé avec cette mesure. On peut alors fournir au modèle une séquence aléatoire plus courte, répétée deux fois, visualiser son pattern d’attention et l’observer directement — y compris la fameuse **« bande d’induction »** à `seq_len - 1` tokens en arrière.

Cette fois, nous plaçons un hook sur l’activation du **pattern d’attention** afin de visualiser le pattern de la tête concernée.


In [44]:
induction_head_layer = 5
induction_head_index = 5
size = (1, 20)
input_tensor = torch.randint(1000, 10000, size)

single_random_sequence = input_tensor.to(model.cfg.device)
repeated_random_sequence = einops.repeat(single_random_sequence, "batch seq_len -> batch (2 seq_len)")
def visualize_pattern_hook(
    pattern: Float[torch.Tensor, "batch head_index dest_pos source_pos"],
    hook: HookPoint,
):
    display(
        cv.attention.attention_patterns(
            tokens=model.to_str_tokens(repeated_random_sequence),
            attention=pattern[0, induction_head_index, :, :][None, :, :] # Add a dummy axis, as CircuitsVis expects 3D patterns.
        )
    )

model.run_with_hooks(
    repeated_random_sequence,
    return_type=None,
    fwd_hooks=[(
        utils.get_act_name("pattern", induction_head_layer),
        visualize_pattern_hook
    )]
)

## Modèles disponibles

TransformerLens est fourni avec plus de 40 modèles open source disponibles, qui peuvent tous être chargés dans une architecture relativement uniforme simplement en changeant le nom dans `from_pretrained`. Les modèles open source disponibles sont [documentés ici](https://dynalist.io/d/n2ZWtnoYHrU1s4vnFSAQ519J#z=jHj79Pj58cgJKdq4t-ygK-4h), et un ensemble de modèles « adaptés à l’interprétabilité » que j’ai entraînés sont [documentés ici](https://dynalist.io/d/n2ZWtnoYHrU1s4vnFSAQ519J#z=NCJ6zH_Okw_mUYAwGnMKsj2m). Cet ensemble inclut de petits modèles jouets (de une à quatre couches) ainsi qu’une série de [modèles SoLU](https://dynalist.io/d/n2ZWtnoYHrU1s4vnFSAQ519J#z=FZ5W6GGcy6OitPEaO733JLqf) allant jusqu’à GPT-2 Medium (300 M de paramètres). Vous pouvez consulter [un tableau des alias officiels et des hyperparamètres des modèles disponibles ici](https://github.com/TransformerLensOrg/TransformerLens/blob/main/transformer_lens/model_properties_table.md).

**Remarque :** TransformerLens ne prend actuellement pas en charge les modèles multi-GPU (nécessaires par exemple pour les modèles au-dessus de \~7B paramètres), mais cette fonctionnalité arrivera bientôt !

Concrètement, cela signifie qu’on peut relancer presque immédiatement la même analyse sur un autre modèle en ne changeant que son nom. Pour le voir, chargeons **DistilGPT-2** (une version distillée de GPT-2, avec deux fois moins de couches) et recopions le code ci-dessus pour observer les **têtes d’induction** dans ce modèle.

In [45]:
# NBVAL_IGNORE_OUTPUT
distilgpt2 = HookedTransformer.from_pretrained("distilgpt2", device=device)

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model distilgpt2 into HookedTransformer


In [46]:
# We make a tensor to store the induction score for each head. We put it on the model's device to avoid needing to move things between the GPU and CPU, which can be slow.
distilgpt2_induction_score_store = torch.zeros((distilgpt2.cfg.n_layers, distilgpt2.cfg.n_heads), device=distilgpt2.cfg.device)

def induction_score_hook(
    pattern: Float[torch.Tensor, "batch head_index dest_pos source_pos"],
    hook: HookPoint,
):
    """
    To document.
    """
    # We take the diagonal of attention paid from each destination position to source positions seq_len-1 tokens back
    # (This only has entries for tokens with index>=seq_len)
    induction_stripe = pattern.diagonal(dim1=-2, dim2=-1, offset=1-seq_len)
    # Get an average score per head
    induction_score = einops.reduce(induction_stripe, "batch head_index position -> head_index", "mean")
    # Store the result.
    distilgpt2_induction_score_store[hook.layer(), :] = induction_score
# end induction_score_hook

# We make a boolean filter on activation names, that's true only on attention pattern names.
pattern_hook_names_filter = lambda name: name.endswith("pattern")

distilgpt2.run_with_hooks(
    repeated_tokens,
    return_type=None, # For efficiency, we don't need to calculate the logits
    fwd_hooks=[(
        pattern_hook_names_filter,
        induction_score_hook
    )]
)

imshow(
    distilgpt2_induction_score_store,
    xaxis="Head",
    yaxis="Layer",
    title="Induction Score by Head in Distil GPT-2"
)

### Aperçu des modèles open source importants présents dans la bibliothèque

* **GPT-2** – les modèles génératifs pré-entraînés classiques d’OpenAI
  * Tailles : Small (85 M), Medium (300 M), Large (700 M) et XL (1,5 B).
  * Entraînés sur \~22 milliards de tokens de texte issu d’internet. ([Réplication open source](https://huggingface.co/datasets/openwebtext))
* **GPT-Neo** – la réplication de GPT-2 par Eleuther
  * Tailles : 125 M, 1,3 B, 2,7 B
  * Entraînés sur \~300 milliards de tokens du [Pile](https://pile.eleuther.ai/), un large jeu de données divers incluant du code (et des contenus atypiques).
* **[OPT](https://ai.facebook.com/blog/democratizing-access-to-large-scale-language-models-with-opt-175b/)** – la série de modèles open source de Meta AI
  * Entraînés sur 180 milliards de tokens de textes variés.
  * Tailles : 125 M, 1,3 B, 2,7 B, 6,7 B, 13 B, 30 B, 66 B
* **GPT-J** – modèle de 6 B paramètres d’Eleuther, entraîné sur le Pile
* **GPT-NeoX** – modèle de 20 B paramètres d’Eleuther, entraîné sur le Pile
* **StableLM** – modèles de Stability AI (3 B et 7 B), avec et sans fine-tuning “chat” et “instruction”
* **Modèles Stanford CRFM** – réplications de GPT-2 Small et GPT-2 Medium, entraînées avec 5 seeds différents.
  * À noter : 600 checkpoints ont été sauvegardés durant l’entraînement de chaque modèle, et ils sont disponibles dans la bibliothèque, par ex. avec `HookedTransformer.from_pretrained("stanford-gpt2-small-a", checkpoint_index=265)`.
* **BERT** – le transformer encodeur bidirectionnel de Google.
  * Taille Base (108 M), entraîné sur Wikipedia anglais et BooksCorpus.

### Aperçu de quelques modèles « interprétabilité-friendly » que j’ai entraînés et inclus

(N’hésitez pas à [me contacter](mailto:neelnanda27@gmail.com) si vous voulez plus de détails sur l’un de ces modèles.)

Chacun de ces modèles a environ **200 checkpoints** sauvegardés durant l’entraînement, qui peuvent également être chargés depuis TransformerLens grâce à l’argument `checkpoint_index` de `from_pretrained`.

À noter que tous les modèles sont entraînés avec un **token de début de séquence (BOS)**, et risquent de ne pas fonctionner correctement si on ne leur en fournit pas !

* **Toy Models** : Inspirés par [A Mathematical Framework](https://transformer-circuits.pub/2021/framework/index.html), j’ai entraîné 12 très petits modèles de langage, de 1 à 4 couches et chacun de largeur 512. Je pense que les interpréter est beaucoup plus accessible que pour les grands modèles, et qu’ils servent à la fois de bonne pratique et de terrain d’expérimentation pour découvrir des motifs et circuits qui se généralisent à des modèles bien plus grands (comme les *induction heads*) :
  * Modèles *Attention-Only* (sans MLP) : attn-only-1l, attn-only-2l, attn-only-3l, attn-only-4l
  * Modèles *GELU* (avec MLP et activation GELU standard) : gelu-1l, gelu-2l, gelu-3l, gelu-4l
  * Modèles *SoLU* (avec MLP et [l’activation SoLU d’Anthropic](https://transformer-circuits.pub/2022/solu/index.html), conçue pour rendre les neurones MLP plus interprétables) : solu-1l, solu-2l, solu-3l, solu-4l
  * Tous ces modèles sont entraînés sur 22 milliards de tokens, composés à 80 % de C4 (web text) et 20 % de code Python.
  * Les modèles ayant le même nombre de couches ont été entraînés avec la même initialisation de poids et le même ordre de données, afin de comparer directement l’effet des fonctions d’activation.
* **Modèles SoLU** : Un scan de modèles plus grands, entraînés avec l’activation SoLU d’Anthropic, dans l’idée de rendre les MLP plus interprétables.
  * Un scan jusqu’à la taille GPT-2 Medium, entraîné sur 30 milliards de tokens du même dataset que les *toy models* (80 % C4, 20 % code Python) :
    * solu-6l (40 M), solu-8l (100 M), solu-10l (200 M), solu-12l (340 M)
  * Un scan plus ancien, également jusqu’à GPT-2 Medium, mais entraîné sur 15 milliards de tokens du [Pile](https://pile.eleuther.ai/) :
    * solu-1l-pile (13 M), solu-2l-pile (13 M), solu-4l-pile (13 M), solu-6l-pile (40 M), solu-8l-pile (100 M), solu-10l-pile (200 M), solu-12l-pile (340 M)

